v2

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
from IPython.display import clear_output

def read_courses():
  f1 = open("course_input.txt",'r')
  courses = {}
  for i in f1.readlines():
    target_course, target_section = i.strip('\n').split()
    courses[target_course] = target_section
  f1.close()
  return courses

def get_table_data(url):
  #Scraping the website HTML
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  table = soup.find("table", attrs={"id" :"customers"})

  # Headers:
    # *nbsp*
    # Course code
    # Program
    # Faculty
    # Credit
    # Section
    # Day, Time, Room
    # Total Seat
    # Seat Booked
    # Seat Remaining

  tr = table.find_all("tr")
  return tr

def check_seat_availability(courses, table_data):
  available = []
  for row in table_data:
    td = row.find_all("td")
    #Checking course, section, and remaining seats
    for course in courses.keys():
      if td[1].text == course and td[5].text == courses[course] and int(td[9].text) > 0:
        data = [i.text.strip() for i in td]
        available.append(data)

  return available

def display_results(available, headers):
  clear_output(wait=False)
  if available:
    print('Available Courses:')
    for course in available:
      print("#===================================#")
      print("Current Time:", time.strftime("%H:%M:%S", time.localtime()), "(UTC)")
      print(headers[0] + ':', course[1])
      print(headers[1] + ':', course[5])
      print(headers[2] + ':', course[9])
      print("#===================================#")
  else:
    print("No seats currently available...")

def periodic_check(interval, url, courses):
  headers = ['Course', 'Section', 'Available Seats']
  while True:
    table_data = get_table_data(url)
    available = check_seat_availability(courses, table_data)
    display_results(available, headers)
    time.sleep(90)

def main():
  url = "https://usis.bracu.ac.bd/academia/admissionRequirement/getAvailableSeatStatus"
  courses = read_courses()
  interval = 90 #seconds
  periodic_check(interval, url, courses)

main()

Available Courses:
#===================================#
Current Time: 06:41:27 (UTC)
Course: CSE251
Section: 05
Available Seats: 1
#===================================#
